<a href="https://colab.research.google.com/github/serereuk/textcnn-for-colab/blob/master/textcnn_for_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install PyDrive

    100% |████████████████████████████████| 993kB 19.3MB/s 
  Running setup.py bdist_wheel for PyDrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pickle

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({"id": "1mrfmLq7mdc8NTMdLMUIOBLNJyX3K4CV8"}) #이부분 수정하면 됩니당
download.GetContentFile("month_tokenize_data.pickle")

In [0]:
with open("month_tokenize_data.pickle","rb") as f:
  data = pickle.load(f)

In [0]:

import time
import datetime
import tensorflow as tf
import numpy as np

class textcnn():
    def __init__(self, sequence_length, num_classes, vocab_size, embedding_size, filter_sizes, num_filters,
                 l2_reg_lambda = 0.0):
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name = "x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name = "y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout")

        l2_loss = tf.constant(0.0)

        with tf.name_scope("embedding"):
            W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name = "W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        pooled_outputs= []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev= 0.1), name= "W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name= "b")
                conv = tf.nn.conv2d(self.embedded_chars_expanded,
                                    W,
                                    strides = [1,1,1,1],
                                    padding="VALID",
                                    name="conv")
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name = "relu")
                pooled = tf.nn.max_pool(h,
                                        ksize = [1, sequence_length - filter_size + 1, 1, 1],
                                        strides= [1,1,1,1],
                                        padding="VALID",
                                        name= "pool")

                pooled_outputs.append(pooled)
        num_filters_total = num_filters * len(filter_sizes)
        #print(pooled_outputs)
        self.h_pool = tf.concat(pooled_outputs,3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
            
        with tf.name_scope("output"):
            W = tf.get_variable("W", shape = [num_filters_total, num_classes]
                                , initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name= "b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="score")
            self.prediction = tf.argmax(self.scores, 1, name="prediction")

        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(labels = self.input_y, logits= self.scores)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        with tf.name_scope("accuracy"):
            correct_prediction = tf.equal(self.prediction, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"), name = "accuracy")








In [0]:
import numpy as np
import tensorflow as tf
import random

def cut(contents, cut= 3):
    results = []
    for content in contents:
        words = content.split()
        result = []
        for word in words:
            result.append(word[:cut])

        results.append("".join([token for token in result]))
    return results

def make_input(documents, max_document):
    vocab = tf.contrib.learn.preprocessing.VocabularyProcessor(max_document)
    x = np.array(list(vocab.fit_transform(documents)))
    vocab_dict = vocab.vocabulary_._mapping
    sorted_vocab = sorted(vocab_dict.items(), key = lambda x: x[1])
    vocabulary = list(list(zip(*sorted_vocab))[0])
    return x, vocabulary, len(vocab.vocabulary_)

def make_output(points, threshold):
    results = np.zeros((len(points),2))
    for idx, point in enumerate(points):
        if point > threshold:
            results[idx,0] = 1
        else:
            results[idx,1] = 1
    return results

def check_maxlength(contents):
    max_document_length = 0
    for document in contents:
        document_length = len(document.split())
        if document_length > max_document_length:
            max_document_length = document_length
    return max_document_length
  
def divide(x, y, train_prop):
    random.seed(1234)
    x = np.array(x)
    y = np.array(y)
    tmp = np.random.permutation(np.arange(len(x)))
    x_tr = x[tmp][:round(train_prop * len(x))]
    y_tr = y[tmp][:round(train_prop * len(x))]
    x_te = x[tmp][-(len(x)-round(train_prop * len(x))):]
    y_te = y[tmp][-(len(x)-round(train_prop * len(x))):]
    return x_tr, x_te, y_tr, y_te


x, vocab, vocab_size = make_input(data['content'], 200)
y = make_output(list(map(int,[x.replace(",","") for x in data['count']])), 100)
x_train, x_test, y_train, y_test = divide(x,y,train_prop=0.8)

In [0]:
import tensorflow as tf

FLAGS = tf.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')
tf.flags.DEFINE_integer("embedding_dim", 128, "for embedding")
tf.flags.DEFINE_string("filter_size", "3,4,5", "filter_size")
tf.flags.DEFINE_integer("num_filters", 128, "number of filter")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "for dropout")
tf.flags.DEFINE_float("l2_reg_lambda", 0.1, "lambda")
tf.flags.DEFINE_integer("batch_size", 64, "batchsize")
tf.flags.DEFINE_integer("num_epochs", 10, "number of training")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate")
tf.flags.DEFINE_integer("checkpoint_every", 100, "save")
tf.flags.DEFINE_integer("num_checkpoints", 5, "number of checkpoints")
tf.flags.DEFINE_boolean("allow_soft_placement", True, "allow soft device")
tf.flags.DEFINE_boolean("log_device_placement", False, "log placement")


INFO:tensorflow:Summary name embedding/W:0 is illegal; using embedding/W_0 instead.
INFO:tensorflow:Summary name embedding/W:0 is illegal; using embedding/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0 is illegal; using conv-maxpool-3/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0 is illegal; using conv-maxpool-3/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0 is illegal; using conv-maxpool-3/b_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0 is illegal; using conv-maxpool-3/b_0 instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0 is illegal; using conv-maxpool-4/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0 is illegal; using conv-maxpool-4/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0 is illegal; using conv-maxpool-4/b_0 instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0 is illegal; using conv-maxpool-4/b_0 instead.
INFO:tensorflow:Summary name conv-maxpool-5/W:0 is illegal; using conv-maxpool-5

KeyboardInterrupt: ignored

In [0]:

with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        cnn = textcnn(sequence_length=x_train.shape[1],
                      num_classes=y_train.shape[1],
                      vocab_size=vocab_size,
                      embedding_size=FLAGS.embedding_dim,
                      filter_sizes=list(map(int, FLAGS.filter_size.split(","))),
                      num_filters=FLAGS.num_filters,
                      l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)


        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)


        def batch_iter(data, batch_size, num_epochs, shuffle=True):
            """
            Generates a batch iterator for a dataset.
            """
            data = np.array(data)
            data_size = len(data)
            num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
            for epoch in range(num_epochs):
                # Shuffle the data at each epoch
                if shuffle:
                    shuffle_indices = np.random.permutation(np.arange(data_size))
                    shuffled_data = data[shuffle_indices]
                else:
                    shuffled_data = data
                for batch_num in range(num_batches_per_epoch):
                    start_index = batch_num * batch_size
                    end_index = min((batch_num + 1) * batch_size, data_size)
                    yield shuffled_data[start_index:end_index]


        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)

        testpoint = 0
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                if testpoint + 100 < len(x_test):
                    testpoint += 100
                else:
                    testpoint = 0
                print("\nEvaluation:")
                dev_step(x_test[testpoint:testpoint+100], y_test[testpoint:testpoint+100], writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

In [0]:
x_train.shape

(5601, 200)

In [0]:
x.shape[1]

200

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
